In [ ]:
#ADJUSTABLE PARAMETERS THAT CHANGE RUN SPEED
minimum_playlist_inclusion = 300 # the minimum number of playlists a song has to be in to be considered for model output
playlist_subset = 10000 # subset of playlist to train on - maximum of 1,000,000

Loading up the song embeddings.

In [2]:
import pickle

with open('../data_storage/CBOW_run_1M_min_5_PP@2024-04-26-10-27-43_con5_pl1000000_emb64_ep1-track2idx.pkl', 'rb') as f:
    data_uris = pickle.load(f)

In [3]:
import torch

# Load the model
model_data = torch.load('../data_storage/CBOW_run_1M_min_5_PP@2024-04-26-10-27-43_con5_pl1000000_emb64_ep1.pt', map_location=torch.device('cpu'))

# Access embeddings and layers
embeddings = model_data['embedding.weight']

In [4]:
# Creating a dictionary of song uri's and their corresponding embeddings
embeddings_dict = {}

for uri, idx in iter(data_uris.items()):
    embeddings_dict[uri] = embeddings[idx]


In [5]:
# Average embedding calculated to use as PAD embedding

average_embedding = torch.mean(embeddings, dim=0)
average_embedding.unsqueeze(0)

tensor([[-0.2239, -0.1767, -0.1768,  0.0902,  0.1923,  0.0818, -0.1311, -0.2306,
         -0.1818, -0.2775,  0.1100,  0.2362,  0.1705, -0.1840,  0.2217,  0.1891,
         -0.1653, -0.2227, -0.2156, -0.2622,  0.1852, -0.2316, -0.2309, -0.2677,
         -0.2711,  0.1375,  0.2533, -0.2247, -0.2740, -0.2182, -0.1831, -0.0821,
         -0.2757, -0.0676, -0.1172,  0.1671, -0.1226, -0.2171, -0.0354,  0.2321,
         -0.2044,  0.0036,  0.2123,  0.2110, -0.1658, -0.2394,  0.2912,  0.1897,
         -0.2654, -0.1867, -0.1602,  0.1672, -0.2275, -0.1471, -0.2599,  0.1432,
         -0.2142,  0.2324,  0.2473, -0.0061,  0.1438,  0.1202,  0.2522,  0.0536]])

In [6]:

import sqlite3
import pandas as pd
import ast

"""conn = sqlite3.connect('../data_storage/spotify.db')

sql_query = '''
    SELECT 
        playlist_track.pid, 
        playlist.playlist_name, 
        artist.artist_uri, 
        track.track_uri,
        playlist.num_albums,
        playlist.num_tracks,
        playlist.num_artists
    FROM playlist_track
    JOIN track ON playlist_track.track_uri = track.track_uri
    JOIN album ON track.album_uri = album.album_uri
    JOIN artist ON album.artist_uri = artist.artist_uri
    JOIN playlist ON playlist_track.pid = playlist.pid
'''

df = pd.read_sql_query(sql_query, conn)

grouped = df.groupby(['pid', 'playlist_name', 'num_albums', 'num_tracks', 'num_artists']).agg({
    'artist_uri': lambda x: list(x),
    'track_uri': lambda x: list(x)
}).reset_index()

grouped.rename(columns={'artist_uri': 'artists', 'track_uri': 'songs'}, inplace=True)

conn.close()

grouped.to_csv('back_up.csv', index=False)

read_from_csv = False"""

grouped = pd.read_csv('back_up.csv')
read_from_csv = True

if read_from_csv:
    grouped['artists'] = grouped['artists'].apply(ast.literal_eval)
    grouped['songs'] = grouped['songs'].apply(ast.literal_eval)


Now need to create embeddings for artist_uri's and playlist titles so that they can be used within the model

In [7]:
"""from gensim.models import Word2Vec # broken taking way too long rn

vector_size = 64
window = 5
min_count = 1
sg = 1  

artist_uris = grouped['artists'].tolist()

model = Word2Vec(sentences=artist_uris, vector_size=vector_size, window=window, min_count=min_count, sg=sg)

artist_embeddings = {}

for artist_uri in model.wv.index_to_key:
    embedding_vector = model.wv[artist_uri]
    artist_embeddings[artist_uri] = embedding_vector"""


"from gensim.models import Word2Vec # broken taking way too long rn\n\nvector_size = 64\nwindow = 5\nmin_count = 1\nsg = 1  \n\nartist_uris = grouped['artists'].tolist()\n\nmodel = Word2Vec(sentences=artist_uris, vector_size=vector_size, window=window, min_count=min_count, sg=sg)\n\nartist_embeddings = {}\n\nfor artist_uri in model.wv.index_to_key:\n    embedding_vector = model.wv[artist_uri]\n    artist_embeddings[artist_uri] = embedding_vector"

In [8]:
"""from transformers import RobertaTokenizer, RobertaModel
import torch
import numpy as np

playlist_names = grouped['playlist_name'].tolist()
playlist_ids = grouped['pid'].tolist()

# Preprocess playlist names to replace invalid values with empty string ""
num_replaced = sum(1 for name in playlist_names if not isinstance(name, str))
print(f"Number of non-string values replaced: {num_replaced}")
playlist_names = [name if isinstance(name, str) else "" for name in playlist_names]

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

# Tokenize all playlist names at once
inputs = tokenizer(playlist_names, return_tensors='pt', padding=True, truncation=True, max_length=512)

# Create batches of inputs
batch_size = 16  # You can adjust this value based on your system's memory capacity
num_samples = len(playlist_names)
num_batches = (num_samples + batch_size - 1) // batch_size

# Create playlist_embeddings dictionary
playlist_embeddings = {}

# Get the default embedding 
default_embedding = np.zeros((model.config.hidden_size,))  

# Process batches
for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, num_samples)

    batch_inputs = {key: value[start_idx:end_idx] for key, value in inputs.items()}

    # Forward pass
    with torch.no_grad():
        outputs = model(**batch_inputs)

    last_hidden_states = outputs.last_hidden_state

    # Calculate mean embeddings
    batch_embeddings = torch.mean(last_hidden_states, dim=1).numpy()

    # Store embeddings in playlist_embeddings dictionary
    for idx, pid in enumerate(playlist_ids[start_idx:end_idx]):
        playlist_embeddings[pid] = batch_embeddings[idx]

print("Embeddings calculated for all playlists.")"""




'from transformers import RobertaTokenizer, RobertaModel\nimport torch\nimport numpy as np\n\nplaylist_names = grouped[\'playlist_name\'].tolist()\nplaylist_ids = grouped[\'pid\'].tolist()\n\n# Preprocess playlist names to replace invalid values with empty string ""\nnum_replaced = sum(1 for name in playlist_names if not isinstance(name, str))\nprint(f"Number of non-string values replaced: {num_replaced}")\nplaylist_names = [name if isinstance(name, str) else "" for name in playlist_names]\n\ntokenizer = RobertaTokenizer.from_pretrained(\'roberta-base\')\nmodel = RobertaModel.from_pretrained(\'roberta-base\')\n\n# Tokenize all playlist names at once\ninputs = tokenizer(playlist_names, return_tensors=\'pt\', padding=True, truncation=True, max_length=512)\n\n# Create batches of inputs\nbatch_size = 16  # You can adjust this value based on your system\'s memory capacity\nnum_samples = len(playlist_names)\nnum_batches = (num_samples + batch_size - 1) // batch_size\n\n# Create playlist_embe

In [9]:
'''import pickle

# File path to save the pickle file
pickle_file_path = "artist_embeddings.pkl"

# Save the artist_embeddings dictionary to a pickle file
with open(pickle_file_path, "wb") as f:
    pickle.dump(artist_embeddings, f)

print("Artist embeddings saved to:", pickle_file_path)'''

'import pickle\n\n# File path to save the pickle file\npickle_file_path = "artist_embeddings.pkl"\n\n# Save the artist_embeddings dictionary to a pickle file\nwith open(pickle_file_path, "wb") as f:\n    pickle.dump(artist_embeddings, f)\n\nprint("Artist embeddings saved to:", pickle_file_path)'

In [10]:
'''file_path = "playlist_name_embeddings.pkl"

# Store the playlist_name_embeddings dictionary as a pickle file
with open(file_path, 'wb') as f:
    pickle.dump(playlist_embeddings, f)

print(f"Playlist name embeddings saved to {file_path}.")'''

'file_path = "playlist_name_embeddings.pkl"\n\n# Store the playlist_name_embeddings dictionary as a pickle file\nwith open(file_path, \'wb\') as f:\n    pickle.dump(playlist_embeddings, f)\n\nprint(f"Playlist name embeddings saved to {file_path}.")'

In [11]:
import pandas as pd
import numpy as np
import pickle

# Load artist embeddings from pickle file
with open("../data_storage/artist_embeddings.pkl", "rb") as f:
    artist_embeddings = pickle.load(f)

# Load playlist name embeddings from pickle file
with open("../data_storage/playlist_name_embeddings.pkl", "rb") as f:
    playlist_embeddings = pickle.load(f)

# Function to calculate average embedding for a list of embeddings
def calculate_average_embedding(embedding_list):
    if embedding_list:
        return np.mean(embedding_list, axis=0)
    else:
        return np.zeros_like(next(iter(artist_embeddings.values())))


save the embeddings to files and upload them...

In [69]:
import torch
import random
from sklearn.model_selection import train_test_split

k_values = [1,5,10,25,50]
# Assuming grouped is your DataFrame containing the data
# first clean dataset and remove playlists with < 15 songs
filtered_grouped = grouped[grouped['songs'].apply(len) >= 20]

subset_size = playlist_subset
random_subset = filtered_grouped.sample(n=subset_size, random_state=42)

# Split the data into training and testing sets
train_df, test_df = train_test_split(random_subset, test_size=0.2, random_state=42)

# Verify the size of training and testing sets
print("Training set size:", len(train_df))
print("Testing set size:", len(test_df))

# Modify training set to use average song embedding, average artist embedding, and playlist name embedding
# Modify training set to use average song embedding, average artist embedding, and playlist name embedding
training_embeddings = []  # Holds the embeddings of k songs in the playlist, average artist embedding, average song embedding, playlist name embedding, and additional features
training_labels = []  # Holds the remaining songs in the playlist that are to be predicted
for _, row in train_df.iterrows():
    p_songs = row['songs']
    k = random.choice(k_values)
    if len(p_songs) >= k:
        selected_songs = random.sample(p_songs, k)
        # Calculate average song embedding only over the selected k songs
        selected_song_embeddings = [embeddings_dict.get(song_uri, []) for song_uri in selected_songs]
        average_song_embedding = np.mean([song_embedding for song_embedding_list in selected_song_embeddings for song_embedding in song_embedding_list], axis=0)
        # Calculate average artist embedding only over the artists of the selected k songs
        selected_artists = [artist_uri for song_uri in selected_songs for artist_uri in row['artists']]
        average_artist_embedding = calculate_average_embedding([artist_embeddings.get(artist_uri, []) for artist_uri in selected_artists])
        p_embeddings = [
            np.atleast_1d(average_song_embedding),  # Ensure at least one-dimensional
            np.atleast_1d(average_artist_embedding),  # Ensure at least one-dimensional
            playlist_embeddings.get(row['pid'], np.zeros_like(next(iter(playlist_embeddings.values())))),  # Playlist name embedding
            row[['num_albums', 'num_tracks', 'num_artists']].values  # Additional features as a single array
        ]
        training_embeddings.append(np.concatenate(p_embeddings))
        # Convert remaining songs to a list of strings
        remaining_songs = [str(song) for song in p_songs if song not in selected_songs]
        training_labels.append(remaining_songs)

# Convert the training data to PyTorch tensors
training_tensor = torch.tensor(training_embeddings, dtype=torch.float32)
# Convert the training labels to a list of lists of strings
# Convert the training labels to a list of lists of strings
training_labels_tensor = [labels for labels in training_labels]

# Modify testing set similarly
# Modify testing set similarly
testing_embeddings = []  # Holds the embeddings of k songs in the playlist, average artist embedding, average song embedding, playlist name embedding, and additional features
testing_labels = []  # Holds the remaining songs in the playlist that are to be predicted
for _, row in test_df.iterrows():
    p_songs = row['songs']
    if len(p_songs) >= k:
        selected_songs = random.sample(p_songs, k)
        # Calculate average song embedding only over the selected k songs
        selected_song_embeddings = [embeddings_dict.get(song_uri, []) for song_uri in selected_songs]
        average_song_embedding = np.mean([song_embedding for song_embedding_list in selected_song_embeddings for song_embedding in song_embedding_list], axis=0)
        # Calculate average artist embedding only over the artists of the selected k songs
        selected_artists = [artist_uri for song_uri in selected_songs for artist_uri in row['artists']]
        average_artist_embedding = calculate_average_embedding([artist_embeddings.get(artist_uri, []) for artist_uri in selected_artists])
        p_embeddings = [
            np.atleast_1d(average_song_embedding),  # Ensure at least one-dimensional
            np.atleast_1d(average_artist_embedding),  # Ensure at least one-dimensional
            playlist_embeddings.get(row['pid'], np.zeros_like(next(iter(playlist_embeddings.values())))),  # Playlist name embedding
            row[['num_albums', 'num_tracks', 'num_artists']].values  # Additional features as a single array
        ]
        testing_embeddings.append(np.concatenate(p_embeddings))
        # Convert remaining songs to a list of strings
        remaining_songs = [str(song) for song in p_songs if song not in selected_songs]
        testing_labels.append(remaining_songs)

# Convert the testing data to PyTorch tensors
testing_tensor = torch.tensor(testing_embeddings, dtype=torch.float32)
# Convert the testing labels to a list of lists of strings
# Convert the training labels to a list of lists of strings
testing_labels_tensor = [labels for labels in testing_labels]







Training set size: 20000
Testing set size: 5000


c:\Apps\Anaconda3\envs\gpu-cuda11\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Apps\Anaconda3\envs\gpu-cuda11\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Apps\Anaconda3\envs\gpu-cuda11\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Apps\Anaconda3\envs\gpu-cuda11\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Apps\Anaconda3\envs\gpu-cuda11\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Apps\Anaconda3\envs\gpu-cuda11\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar 

In [70]:
import numpy as np

def r_precision(y_true, y_pred):
    # Convert y_true to a set for quick lookup -- Probably not needed
    y_true_set = set(y_true)

    # Determine the number of relevant items -- Not 100% sure
    r = len(y_true_set)
    
    # Filter y_pred to only include items that are in y_true -- Not 100% sure if this is correct
    relevant_predictions = [song for song in y_pred if song in y_true_set]
    
    # Calculate the number of relevant items that should be considered
    r_actual = min(r, len(relevant_predictions))
    
    # Count how many of the top-r_actual predictions are in the true set of relevant songs
    relevant_count = len(relevant_predictions[:r_actual])
    
    if r == 0:
        return 0  # Avoid division by zero if there are no relevant items
    return relevant_count / r

def recommended_songs_clicks(y_true, y_pred):
    # Convert ground truth list to set for quick look-up -- Again, probably not needed
    true_set = set(y_true)
    
    # Find the first relevant track in the predictions
    for i, track in enumerate(y_pred, start=1):
        if track in true_set:
            # how many blocks of 10 tracks are needed
            return (i - 1) // 10 + 1
    
    # If no relevant track is found, set a default value
    # Since the max number of clicks possible plus one is mentioned as 51
    return 51


def dcg(relevances, rank):
    relevances = np.array(relevances)
    if relevances.size:
        return relevances[0] + np.sum(relevances[1:] / np.log2(np.arange(2, relevances.size + 1)))
    return 0

def idcg(relevances):
    sorted_relevances = sorted(relevances, reverse=True)
    return dcg(sorted_relevances, len(sorted_relevances))

def ndcg(y_true, y_pred):
    relevances = [1 if song in y_true else 0 for song in y_pred]
    actual_dcg = dcg(relevances, len(relevances))
    ideal_dcg = idcg(relevances)
    if ideal_dcg == 0:
        return 0
    return actual_dcg / ideal_dcg


def playlist_loss(y_true, y_pred):
    total_loss = 0
    num_samples = len(y_true)
    
    for i in range(num_samples):
        # Convert y_true and y_pred to sets for quick lookup
        y_true_set = set(y_true[i])
        y_pred_set = set(y_pred[i])

        # Calculate R-Precision
        r_precision_value = r_precision(y_true_set, y_pred_set)

        # Calculate NDCG
        ndcg_value = ndcg(y_true_set, y_pred_set)

        # Calculate clicks
        clicks_value = recommended_songs_clicks(y_true_set, y_pred_set)

        # Define weights
        weight_r = 1
        weight_ndcg = 1
        weight_clicks = 1/50  # Adjusted weight for clicks

        # Calculate individual losses
        r_loss = 1 - r_precision_value
        ndcg_loss = 1 - ndcg_value

        # Subtract 1/50 to keep loss within the range of 0 to 3, as 1 click (1/50) is the minimum
        clicks_loss = clicks_value * weight_clicks - 1/50

        # Combine individual losses
        loss = (r_loss * weight_r) + (ndcg_loss * weight_ndcg) + clicks_loss

        # Accumulate total loss
        total_loss += loss
        if (loss>3):
            print(f"loss: {loss}, r_prec: {r_precision_value}, ndcg: {ndcg_value}, clicks: {clicks_value} ")
    
    # Calculate average loss
    average_loss = total_loss / num_samples
    print(average_loss)
    
    return average_loss





In [71]:
# Initialize an empty dictionary to store the embeddings of songs that appear in more than 5 playlists
filtered_embeddings_dict = {}

# Calculate playlist counts for each song URI
playlist_counts = {}
for songs_list in grouped['songs']:
    for song_uri in songs_list:
        playlist_counts[song_uri] = playlist_counts.get(song_uri, 0) + 1

# Filter embeddings dictionary to include only songs that appear in more than 5 playlists
for song_uri, embedding in embeddings_dict.items():
    if playlist_counts.get(song_uri, 0) > minimum_playlist_inclusion:
        filtered_embeddings_dict[song_uri] = embedding


In [72]:
print(len(embeddings_dict))
print(len(filtered_embeddings_dict))

524234
13157


In [73]:
import torch
import torch.nn as nn
import torch.optim as optim


# Move model and tensors to CUDA device if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define your model using PyTorch
class CustomModel(nn.Module):
    def __init__(self, input_size):
        super(CustomModel, self).__init__()
        self.input_size = input_size
        self.dense1 = nn.Linear(input_size, 128)  # Adjust the input size of the first layer
        self.dense2 = nn.Linear(128, 64)

    def forward(self, x):
        x = torch.relu(self.dense1(x))
        x = torch.relu(self.dense2(x))
        return x

# Custom layer to find closest embeddings
class ClosestEmbeddingsLayer(nn.Module):
    def __init__(self, num_closest=500):
        super(ClosestEmbeddingsLayer, self).__init__()
        self.num_closest = num_closest

    def forward(self, generated_embeddings):
        closest_embeddings_batch = []
        for generated_embedding in generated_embeddings:
            # Compute cosine similarity between the generated embedding and all embeddings in embeddings_dict
            similarities = {}
            for key, value in filtered_embeddings_dict.items():
                similarity = torch.nn.functional.cosine_similarity(generated_embedding.unsqueeze(0), value.unsqueeze(0))
                similarities[key] = similarity.item()
            
            # Sort the similarities and get the top num_closest embeddings
            closest_embeddings = sorted(similarities, key=similarities.get, reverse=True)[:self.num_closest]
            closest_embeddings_batch.append(closest_embeddings)
        
        return closest_embeddings_batch

# Create model instance
input_size = training_tensor.shape[1]  # Get input size from the shape of your input tensor
model = CustomModel(input_size).to(device)
closest_embeddings_layer = ClosestEmbeddingsLayer().to(device)

# Define optimizer
optimizer = optim.Adam(model.parameters())

# Training loop
num_epochs = 10
batch_size = 32
for epoch in range(num_epochs):
    model.train()
    for i in range(0, len(training_tensor), batch_size):
        print(i)
        optimizer.zero_grad()
        batch_input = training_tensor[i:i+batch_size].to(device)
        batch_target = training_labels_tensor[i:i+batch_size]  # Use training labels tensor
        output = model(batch_input)
        closest_embeddings = closest_embeddings_layer(output)
        loss = playlist_loss(batch_target, closest_embeddings)
        loss_tensor = torch.tensor(loss, requires_grad=True, device=device)
        loss_tensor.backward()  # Compute gradients
        optimizer.step()  # Update weights

    # Validation
    model.eval()
    val_loss_total = 0.0  # Variable to accumulate the total validation loss
    with torch.no_grad():
        for i in range(0, len(testing_tensor), batch_size):
            batch_input = testing_tensor[i:i+batch_size].to(device)
            batch_target = testing_labels_tensor[i:i+batch_size]  # Use testing labels tensor
            val_output = model(batch_input)
            val_closest_embeddings = closest_embeddings_layer(val_output)
            val_loss = playlist_loss(batch_target, val_closest_embeddings)
            val_loss_total += val_loss.item()

    # Calculate the average validation loss across all batches
    average_val_loss = val_loss_total / (len(testing_tensor) / batch_size)
    print(f"Epoch {epoch+1}, Validation Loss: {average_val_loss:.4f}")


0
2.485584559965364
32
2.4970515113017577
64
2.4761532684100005
96
2.716351021267675
128
2.649735511213517
160
2.579933725586716
192
2.604004992280962
224
2.6289883248473
256
2.5507556351152756
288
2.522665125879232
320
2.6214975132705414
352
2.5677605346457675
384
2.5084092972141177
416
2.567690093393826
448
2.545861654947249
480
2.5544342275330765
512
2.671286935253885
544
2.5062029523127576
576
2.5145359967376213
608
2.61833835832419
640
2.4969283204989803
672
2.548791962749367
704
2.618959087096803
736
2.515353875101538
768
2.44744660649718
800
2.3679162550563553
832
2.5143397780813235
864
2.5372426133588646
896
2.5291330501391633
928
2.622084172073027
960
2.5800705899670584
992
2.5323116628043842
1024
2.4557229486154757
1056
2.5920722466176094
1088
2.5690200378466015
1120
2.560201321321971
1152
2.5005900607816782
1184
2.563400136572032
1216
2.617424411559691
1248
2.4368360719252027
1280
2.6949017260260515
1312
2.5402563826362488
1344
2.3642472583082306
1376
2.5402835728516653
1408

Instead try having n neurons in final layer where each neuron is a song which has had an embedding generated for it??